In [1]:
# Loading library
import os
os.sys.path.append("../")
from scripts.etl_pipeline import *

In [2]:

# Create a Spark Session
spark = (
    SparkSession.builder.appName("ETL Pipeline")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "2g")
    .getOrCreate()
)

24/09/10 18:43:35 WARN Utils: Your hostname, Alistairs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.13.156.189 instead (on interface en0)
24/09/10 18:43:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/10 18:43:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Extract

As we can't use `urlretrieve` to get the data from Canvas, please download it to your local machine and move it `data/tables`. Then run the code below to unzip the files

In [6]:
# Assign data path
raw_path = "../data"

# Unzip files (Only run once)
"""
for file in os.listdir(f"{raw_path}/tables"):
    if file == ".gitkeep":
        continue
    with zipfile.ZipFile(f"{raw_path}/tables/{file}", "r") as zip_ref:
        zip_ref.extractall(f"{raw_path}/")
    os.remove(f"{raw_path}/tables/{file}")
"""

'\nfor file in os.listdir(f"{raw_path}/tables"):\n    if file == ".gitkeep":\n        continue\n    with zipfile.ZipFile(f"{raw_path}/tables/{file}", "r") as zip_ref:\n        zip_ref.extractall(f"{raw_path}/")\n    os.remove(f"{raw_path}/tables/{file}")\n'

# Transform

The system use `user_id` as a key for identifying customer in transactions record and fraud probability tables. However, they also have a key-value map of `user_id` and `consumer_id`. We will use `consumer_id` as the only ID for customer. Thus, we will map `user_id` from each table to `consumer_id` and drop the former.


In [7]:
# Load consumer user details -> a key:value map for user_id to consumer_id
consumer_user_map = spark.read.parquet(f"{raw_path}/tables/consumer_user_details.parquet")
consumer_user_map.limit(5) # Preview

user_id,consumer_id
1,1195503
2,179208
3,1194530
4,154128
5,712975


In [8]:
# Load consumer fraud rate dataset
consumer_fraud_rate = spark.read.csv(f"{raw_path}/tables/consumer_fraud_probability.csv", header=True, inferSchema=True)
consumer_fraud_rate.show(5)
get_dataset_count(consumer_fraud_rate)

# Replace all user_id with unique consumer_id
consumer_fraud_rate = replace_id(consumer_user_map, consumer_fraud_rate)
consumer_fraud_rate.show(5)

# Check to make sure no rows were lost on the inner join
get_dataset_count(consumer_fraud_rate)

+-------+--------------+-----------------+
|user_id|order_datetime|fraud_probability|
+-------+--------------+-----------------+
|   6228|    2021-12-19| 97.6298077657765|
|  21419|    2021-12-10|99.24738020302328|
|   5606|    2021-10-17|84.05825045251777|
|   3101|    2021-04-17|91.42192091901347|
|  22239|    2021-10-19|94.70342477508035|
+-------+--------------+-----------------+
only showing top 5 rows

The dataset count is  34864
+--------------+------------------+-----------+
|order_datetime| fraud_probability|consumer_id|
+--------------+------------------+-----------+
|    2022-02-20| 9.805431136520959|    1195503|
|    2021-08-30| 9.599513915425788|     179208|
|    2021-09-25|10.069850934775245|     179208|
|    2021-11-03| 8.300636455314633|    1194530|
|    2021-10-09| 9.633302411090419|     154128|
+--------------+------------------+-----------+
only showing top 5 rows

The dataset count is  34864


In [9]:
# Load all the transaction data 
transaction_p1 = spark.read.parquet(f"{raw_path}/tables/transactions_20210228_20210827_snapshot")
transaction_p2 = spark.read.parquet(f"{raw_path}/tables/transactions_20210828_20220227_snapshot")
transaction_p3 = spark.read.parquet(f"{raw_path}/tables/transactions_20220228_20220828_snapshot")

# Combine the datasets
transaction_records = reduce(DataFrame.unionAll, [transaction_p1, transaction_p2, transaction_p3])
# transaction_records.show(5)
get_dataset_count(transaction_records)

# Replace user_id with consumer_id after combining
transaction_records = replace_id(consumer_user_map, transaction_records)
transaction_records.show(5)

# Check to make sure no rows were lost on the inner join
get_dataset_count(transaction_records)

The dataset count is  14195505
+------------+------------------+--------------------+--------------+-----------+
|merchant_abn|      dollar_value|            order_id|order_datetime|consumer_id|
+------------+------------------+--------------------+--------------+-----------+
| 62191208634|63.255848959735246|949a63c8-29f7-4ab...|    2021-08-20|     651338|
| 15549624934| 130.3505283105634|6a84c3cf-612a-457...|    2021-08-20|     179208|
| 64403598239|120.15860593212783|b10dcc33-e53f-425...|    2021-08-20|     467663|
| 60956456424| 136.6785200286976|0f09c5a5-784e-447...|    2021-08-20|    1194530|
| 94493496784| 72.96316578355305|f6c78c1a-4600-4c5...|    2021-08-20|     467663|
+------------+------------------+--------------------+--------------+-----------+
only showing top 5 rows



The dataset count is  14195505


## Cleaning

Now that replacing `user_id` to `consumer_id` is done, load all other data and clean them. First, let's have a look at the merchant data on fraud probability.

In [10]:
# Load merchant fraud rate dataset
merchant_fraud_rate = spark.read.csv(f"{raw_path}/tables/merchant_fraud_probability.csv", header=True, inferSchema=True)
merchant_fraud_rate.show(5)
get_dataset_count(merchant_fraud_rate)

+------------+--------------+------------------+
|merchant_abn|order_datetime| fraud_probability|
+------------+--------------+------------------+
| 19492220327|    2021-11-28|44.403658647495355|
| 31334588839|    2021-10-02| 42.75530083865367|
| 19492220327|    2021-12-22|38.867790051131095|
| 82999039227|    2021-12-19|  94.1347004808891|
| 90918180829|    2021-09-02| 43.32551731714902|
+------------+--------------+------------------+
only showing top 5 rows

The dataset count is  114


Cleaning `tbl_merchants.parquet`. The feature `tags` is a string that represents either a tuple or a list, containing 3 elements:
* Items that are being sold
* Revenue levels
* Commission rate

Each elements either a list, a tuple, or a combination of both (e.g starts with `[` and ends with `)` and vice versa). These inconsistencies are mostly due to human errors. Thus, we need to take into account these consistent when splitting the values of the feature `tags` into separate columns

In [11]:
# Load merchant's info
merchant_info = spark.read.parquet(f"{raw_path}/tables/tbl_merchants.parquet")
merchant_info.show(5, truncate=False)

# Clean the data
merchant_info = clean_merchant_details(merchant_info)
merchant_info.show(5, truncate=False)

+------------------------------------+-----------------------------------------------------------------------------------------------------------------+------------+
|name                                |tags                                                                                                             |merchant_abn|
+------------------------------------+-----------------------------------------------------------------------------------------------------------------+------------+
|Felis Limited                       |((furniture, home furnishings and equipment shops, and manufacturers, except appliances), (e), (take rate: 0.18))|10023283211 |
|Arcu Ac Orci Corporation            |([cable, satellite, and otHer pay television and radio services], [b], [take rate: 4.22])                        |10142254217 |
|Nunc Sed Company                    |([jewelry, watch, clock, and silverware shops], [b], [take rate: 4.40])                                          |10165489824 |
|Ult

The data on consumer's basic information is a single column that contains the consumer's name, address, state, postcode, gender, and their unqiue consumer ID, each separated by "`|`". Thus, we will need to split these into individual columns. Based on the `README.md` for the data, we will only keep the consumer's name, state, postcode, gender, and consumer ID as the addresses are fake.

In [12]:
# Load consumer info's
consumer_info = spark.read.csv(f"{raw_path}/tables/tbl_consumer.csv", header=True, inferSchema=True)
consumer_info.show(5, truncate=False)

# Clean the data
consumer_info = clean_consumer_details(consumer_info)
consumer_info.limit(5)

+---------------------------------------------------------------------+
|name|address|state|postcode|gender|consumer_id                       |
+---------------------------------------------------------------------+
|Yolanda Williams|413 Haney Gardens Apt. 742|WA|6935|Female|1195503   |
|Mary Smith|3764 Amber Oval|NSW|2782|Female|179208                    |
|Jill Jones MD|40693 Henry Greens|NT|862|Female|1194530               |
|Lindsay Jimenez|00653 Davenport Crossroad|NSW|2780|Female|154128     |
|Rebecca Blanchard|9271 Michael Manors Suite 651|WA|6355|Female|712975|
+---------------------------------------------------------------------+
only showing top 5 rows

Before: 
The dataset count is  499999
After: 
The dataset count is  499999


name,consumer_id,gender,state,postcode
Yolanda Williams,1195503,Female,WA,6935
Mary Smith,179208,Female,NSW,2782
Jill Jones MD,1194530,Female,NT,862
Lindsay Jimenez,154128,Female,NSW,2780
Rebecca Blanchard,712975,Female,WA,6355


We will also need to ensure that the datetime of all dataset with such column is within the specified range (labeled on the name of the intial downloaded file)

In [14]:
transaction_records = ensure_datetime_range(transaction_records, "2021-02-28", "2022-08-28")

merchant_fraud_rate = ensure_datetime_range(merchant_fraud_rate, "2021-02-08", "2022-08-28")

consumer_fraud_rate = ensure_datetime_range(consumer_fraud_rate, "2021-02-08", "2022-08-28")

Starting entries: 12561377 
Final entries: 12561377
Net change (%): 0.0 
Starting entries: 114 
Final entries: 114
Net change (%): 0.0 
Starting entries: 34864 
Final entries: 34864
Net change (%): 0.0 


Next,  we check for any existing null values across all dataset.

In [15]:
# Check for Missing values
calculate_missing_values(consumer_fraud_rate)
calculate_missing_values(merchant_fraud_rate)
calculate_missing_values(consumer_info)
calculate_missing_values(merchant_info)
calculate_missing_values(transaction_records)

+----------------------------+-------------------------------+-------------------------+
|order_datetime_missing_count|fraud_probability_missing_count|consumer_id_missing_count|
+----------------------------+-------------------------------+-------------------------+
|                           0|                              0|                        0|
+----------------------------+-------------------------------+-------------------------+

+--------------------------+----------------------------+-------------------------------+
|merchant_abn_missing_count|order_datetime_missing_count|fraud_probability_missing_count|
+--------------------------+----------------------------+-------------------------------+
|                         0|                           0|                              0|
+--------------------------+----------------------------+-------------------------------+



+------------------+-------------------------+--------------------+-------------------+----------------------+
|name_missing_count|consumer_id_missing_count|gender_missing_count|state_missing_count|postcode_missing_count|
+------------------+-------------------------+--------------------+-------------------+----------------------+
|                 0|                        0|                   0|                  0|                     0|
+------------------+-------------------------+--------------------+-------------------+----------------------+

+------------------+--------------------------+----------------------+---------------------------+-----------------------+
|name_missing_count|merchant_abn_missing_count|category_missing_count|revenue_level_missing_count|take_rate_missing_count|
+------------------+--------------------------+----------------------+---------------------------+-----------------------+
|                 0|                         0|                     0|     

+--------------------------+--------------------------+----------------------+----------------------------+-------------------------+
|merchant_abn_missing_count|dollar_value_missing_count|order_id_missing_count|order_datetime_missing_count|consumer_id_missing_count|
+--------------------------+--------------------------+----------------------+----------------------------+-------------------------+
|                         0|                         0|                     0|                           0|                        0|
+--------------------------+--------------------------+----------------------+----------------------------+-------------------------+



# Save

We now export the curated into the `data/curated` directory for analysis and modelling.

In [16]:
landing_directory = "../data/curated"

In [ ]:
# Write cleaned data to disk. (Overwrite will replace any existing files)
consumer_fraud_rate.write.mode("overwrite").parquet(f"{landing_directory}/consumer_fraud_prob.parquet")
merchant_fraud_rate.write.mode("overwrite").parquet(f"{landing_directory}/merchant_fraud_prob.parquet")
transaction_records.write.mode("overwrite").parquet(f"{landing_directory}/transactions.parquet")
merchant_info.write.mode("overwrite").parquet(f"{landing_directory}/merchant_info.parquet")
consumer_info.write.mode("overwrite").parquet(f"{landing_directory}/consumer_info.parquet")


In this section, we begin exploratory data analysis to gain a better understanding of the data and our variables

In [ ]:
"""
from pyspark.sql import functions as F

# Revenue analysis
# Explore for each ABN, their total revenue, average order value and total number of orders
merchant_performance = transaction_records.groupBy("merchant_abn").agg(
    F.sum("dollar_value").alias("total_revenue"),
    F.avg("dollar_value").alias("average_order_value"),
    F.count("dollar_value").alias("total_orders")
)

# Rank merchants by total revenue in descending order
merchant_performance = merchant_performance.orderBy(F.col("total_revenue").desc())
merchant_performance.show(10)

get_dataset_count(merchant_performance)
calculate_missing_values(merchant_performance)
"""

In [ ]:
"""
from pyspark.sql.functions import avg

merchant_avg_fraud_prob = merchant_fraud_rate.groupBy("merchant_abn").agg(avg("fraud_probability").alias("merchant_avg_fraud_prob"))

merchant_avg_fraud_prob.show()

calculate_missing_values(merchant_avg_fraud_prob)
"""

In [ ]:
"""
# Risk Analysis
# Combine revenue analysis with fraud probabilities
merchant_performance_risk = merchant_performance.join(merchant_avg_fraud_prob, on="merchant_abn", how="left")

# Rank merchants by fraud probability in descending order
merchant_performance_risk = merchant_performance_risk.orderBy(F.col("merchant_avg_fraud_prob").desc())
merchant_performance_risk.show(10)

get_dataset_count(merchant_performance_risk)
calculate_missing_values(merchant_performance_risk)
"""

In [ ]:
"""
# Profitability analysis
# Combine previous merchant_performance_risk with merchants_info to get take_rate and revenue_level
merchant_profitability = merchant_performance_risk.join(merchant_info, on="merchant_abn", how="inner")

# Calculate net revenue
merchant_profitability = merchant_profitability.withColumn(
    "net_revenue", F.col("total_revenue") * F.col("take_rate")/100
)

# Rank merchants by profitability in descending order
merchant_profitability.orderBy(F.col("net_revenue").desc()).show(10)


# Rank merchants by profitability in ascending order
merchant_profitability.orderBy(F.col("net_revenue").asc()).show(10)

get_dataset_count(merchant_profitability)
calculate_missing_values(merchant_profitability)
"""